In [3]:
!pip install strands-agents strands-agents-tools


  Using cached docstring_parser-0.17.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached anyio-4.10.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached wrapt-1.17.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.4 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached aiohttp-3.12.15-cp313-cp313-macosx_11_0_arm64.whl.metadata (7.7 kB)
  Using cached markdownify-1.2.0-py3-none-any.whl.metadata (9.9 kB)
  Using cached pillow-11.3.0-cp313-cp313-macosx_11_0_arm6

In [ ]:
#https://github.com/strands-agents/sdk-python

#https://strandsagents.com/latest/documentation/docs/user-guide/concepts/model-providers/openai/

In [5]:
from strands import Agent, tool
import sqlite3
import pandas as pd
import os

In [8]:
conn = sqlite3.connect('data/dubai_infrastructure.db')

In [106]:
query = "SELECT * FROM alerts JOIN infrastructure_assets ON alerts.asset_id = infrastructure_assets.asset_id WHERE alerts.severity in ('high', 'critical') AND alerts.status = 'active' ORDER BY alerts.created_at DESC LIMIT 10"
pd.read_sql(query, conn)

,alert_id,asset_id,alert_type,severity,description,created_at,status,zone_id,district,estimated_cost_aed,...,district,zone_id,condition_score,risk_level,last_inspection,daily_usage,maintenance_cost_aed,criticality,construction_year,next_inspection_due
0,ALT_0097,ROAD_254,critical_condition,critical,Critical condition detected: Jumeirah Road Sec...,2025-09-20 22:27:54.145324,active,JM,Jumeirah,93153,...,Jumeirah,JM,88,low,2025-05-30,44576,385377,medium,2000,2025-11-23
1,ALT_0121,UTIL_416,critical_condition,critical,Critical condition detected: Sheikh Zayed Road...,2025-09-20 13:28:00.190615,active,SZR,Sheikh Zayed Road Corridor,179808,...,Sheikh Zayed Road Corridor,SZR,56,high,2025-07-05,49979,460921,medium,2021,2025-12-28
2,ALT_0007,UTIL_152,critical_condition,critical,Critical condition detected: Business Bay Util...,2025-09-20 00:17:45.669168,active,BB,Business Bay,90064,...,Business Bay,BB,76,medium,2025-04-05,52012,111745,low,2006,2025-11-05
3,ALT_0108,UTIL_217,environmental_hazard,high,Environmental conditions affecting Deira Utili...,2025-09-19 07:36:55.390223,active,DR,Deira,111282,...,Deira,DR,56,high,2025-07-02,23202,204117,medium,2000,2025-12-06
4,ALT_0119,ROAD_293,sensor_fault,high,Sensor malfunction detected on Business Bay Ro...,2025-09-16 10:30:46.516972,active,BB,Business Bay,18889,...,Business Bay,BB,59,high,2025-03-26,54816,124928,medium,2009,2025-11-21
5,ALT_0039,BRID_027,maintenance_due,high,Scheduled maintenance required for Al Maktoum ...,2025-09-15 05:38:11.816193,active,DR,Deira,121679,...,Deira,DR,51,high,2025-07-22,75000,1665315,high,1999,2025-11-24
6,ALT_0029,UTIL_302,high_usage,high,Usage levels exceeded normal capacity on Jumei...,2025-09-14 22:04:18.675631,active,JM,Jumeirah,66963,...,Jumeirah,JM,59,high,2025-03-31,31188,390497,medium,2005,2026-01-16
7,ALT_0129,ROAD_266,critical_condition,critical,Critical condition detected: JBR Road Section ...,2025-09-14 12:04:35.317826,active,JBR,JBR,368362,...,JBR,JBR,47,high,2025-07-25,9222,426133,low,2000,2025-11-20
8,ALT_0115,ROAD_158,critical_condition,high,Critical condition detected: Jumeirah Village ...,2025-09-13 11:53:25.566274,active,JVC,Jumeirah Village Circle,23255,...,Jumeirah Village Circle,JVC,78,medium,2025-04-22,57640,303490,low,2015,2025-12-16
9,ALT_0137,ROAD_058,sensor_fault,high,Sensor malfunction detected on Deira Road Sect...,2025-09-13 08:32:09.846541,active,DR,Deira,213643,...,Deira,DR,64,high,2025-08-03,63082,368528,low,2020,2025-12-29


In [ ]:
#Tools
1. Query Infrastructure by district
2. Query the critical alerts
3. Query infrastructure Sats


In [28]:
def get_db_connection(db_path='data/dubai_infrastructure.db'):
    if not os.path.exists(db_path):
        raise FileNotFoundError(f"Database file not found at {db_path}")
    conn = sqlite3.connect(db_path)
    return conn



def query_infrastructure_by_district(district_name: str='all', risk_level: str='all') -> str:
    """
    Query infrastructure details by district name and risk level.
    """
    conn = get_db_connection()

    query = f"SELECT asset_name, asset_type, district, risk_level, daily_usage, maintenance_cost_aed,  construction_year, next_inspection_due, condition_score FROM infrastructure_assets WHERE district LIKE '%{district_name}%' "
    if risk_level != 'all':
        query += f" AND risk_level = '{risk_level}'"

    query += " ORDER BY condition_score ASC LIMIT 10;"
    df = pd.read_sql(query, conn)
    return df.to_string(index=False)

In [29]:
query_infrastructure_by_district(district_name='Deira')

'                asset_name asset_type district risk_level  daily_usage  maintenance_cost_aed  construction_year next_inspection_due  condition_score\n Deira Utility Section 386    utility    Deira       high         6526                417326               2016          2025-12-20               32\nDubai Festival City Access       road    Deira   critical        40000                735795               1999          2025-06-29               39\n     DEWA Substation Deira    utility    Deira   critical       120000                828673               2006          2025-12-10               41\n    Deira Road Section 443       road    Deira       high        28839                406742               2008          2025-11-11               45\n    Deira Road Section 336       road    Deira       high        40200                375827               2022          2026-01-14               46\n    Deira Road Section 269       road    Deira       high        72425                436148       

In [56]:
query = "SELECT * FROM alerts JOIN infrastructure_assets ON alerts.asset_id = infrastructure_assets.asset_id WHERE alerts.severity in ('high', 'critical') AND alerts.status = 'active' AND infrastructure_assets.risk_level = 'critical' ORDER BY alerts.created_at DESC LIMIT 10;"
pd.read_sql(query, conn)

,alert_id,asset_id,alert_type,severity,description,created_at,status,zone_id,district,estimated_cost_aed,...,district,zone_id,condition_score,risk_level,last_inspection,daily_usage,maintenance_cost_aed,criticality,construction_year,next_inspection_due


In [63]:
def get_critical_alerts(risk_level: str='critical') -> str:
    """
    Get critical alerts from the infrastructure assets.
    """
    conn = get_db_connection()
    query = f"SELECT * FROM alerts JOIN infrastructure_assets ON alerts.asset_id = infrastructure_assets.asset_id WHERE alerts.severity in ('high', 'critical') AND alerts.status = 'active' AND infrastructure_assets.risk_level = '{risk_level}' ORDER BY alerts.created_at DESC LIMIT 10;"
    print(query)
    df = pd.read_sql(query, conn)
    return df.to_string(index=False)

In [66]:
get_critical_alerts('high')

SELECT * FROM alerts JOIN infrastructure_assets ON alerts.asset_id = infrastructure_assets.asset_id WHERE alerts.severity in ('high', 'critical') AND alerts.status = 'active' AND infrastructure_assets.risk_level = 'high' ORDER BY alerts.created_at DESC LIMIT 10;


,alert_id,asset_id,alert_type,severity,description,created_at,status,zone_id,district,estimated_cost_aed,...,district,zone_id,condition_score,risk_level,last_inspection,daily_usage,maintenance_cost_aed,criticality,construction_year,next_inspection_due
0,ALT_0121,UTIL_416,critical_condition,critical,Critical condition detected: Sheikh Zayed Road...,2025-09-20 13:28:00.190615,active,SZR,Sheikh Zayed Road Corridor,179808,...,Sheikh Zayed Road Corridor,SZR,56,high,2025-07-05,49979,460921,medium,2021,2025-12-28
1,ALT_0108,UTIL_217,environmental_hazard,high,Environmental conditions affecting Deira Utili...,2025-09-19 07:36:55.390223,active,DR,Deira,111282,...,Deira,DR,56,high,2025-07-02,23202,204117,medium,2000,2025-12-06
2,ALT_0119,ROAD_293,sensor_fault,high,Sensor malfunction detected on Business Bay Ro...,2025-09-16 10:30:46.516972,active,BB,Business Bay,18889,...,Business Bay,BB,59,high,2025-03-26,54816,124928,medium,2009,2025-11-21
3,ALT_0039,BRID_027,maintenance_due,high,Scheduled maintenance required for Al Maktoum ...,2025-09-15 05:38:11.816193,active,DR,Deira,121679,...,Deira,DR,51,high,2025-07-22,75000,1665315,high,1999,2025-11-24
4,ALT_0029,UTIL_302,high_usage,high,Usage levels exceeded normal capacity on Jumei...,2025-09-14 22:04:18.675631,active,JM,Jumeirah,66963,...,Jumeirah,JM,59,high,2025-03-31,31188,390497,medium,2005,2026-01-16
5,ALT_0129,ROAD_266,critical_condition,critical,Critical condition detected: JBR Road Section ...,2025-09-14 12:04:35.317826,active,JBR,JBR,368362,...,JBR,JBR,47,high,2025-07-25,9222,426133,low,2000,2025-11-20
6,ALT_0137,ROAD_058,sensor_fault,high,Sensor malfunction detected on Deira Road Sect...,2025-09-13 08:32:09.846541,active,DR,Deira,213643,...,Deira,DR,64,high,2025-08-03,63082,368528,low,2020,2025-12-29
7,ALT_0140,ROAD_206,sensor_fault,high,Sensor malfunction detected on Jumeirah Road S...,2025-09-05 23:18:07.311886,active,JM,Jumeirah,106905,...,Jumeirah,JM,53,high,2025-05-30,40811,145260,medium,2013,2025-12-11
8,ALT_0120,UTIL_094,maintenance_due,high,Scheduled maintenance required for Sheikh Zaye...,2025-09-04 17:56:51.432777,active,SZR,Sheikh Zayed Road Corridor,175291,...,Sheikh Zayed Road Corridor,SZR,64,high,2025-04-08,8534,286580,low,2021,2025-10-24
9,ALT_0005,ROAD_385,critical_condition,high,Critical condition detected: Sheikh Zayed Road...,2025-09-02 20:46:14.555792,active,SZR,Sheikh Zayed Road Corridor,84846,...,Sheikh Zayed Road Corridor,SZR,56,high,2025-05-09,69629,278554,low,2012,2025-11-02


In [69]:
def get_infrastructure_stats_by_risk() -> str:
    """
    Get infrastructure statistics grouped by risk level.
    """
    conn = get_db_connection()
    query = """
        SELECT risk_level, 
               COUNT(*) AS total_assets, 
               AVG(condition_score) AS avg_condition_score, 
               SUM(maintenance_cost_aed) AS total_maintenance_cost 
        FROM infrastructure_assets 
        GROUP BY risk_level
    """
    df = pd.read_sql(query, conn)
    return df

In [ ]:
def get_infrastructure_stats_by_district() -> str:
    """
    Get infrastructure statistics grouped by district.
    """
    conn = get_db_connection()
    query = """
        SELECT district, 
               COUNT(*) AS total_assets, 
               AVG(condition_score) AS avg_condition_score, 
               SUM(maintenance_cost_aed) AS total_maintenance_cost 
        FROM infrastructure_assets 
        GROUP BY district
    """
    df = pd.read_sql(query, conn)
    return df

In [80]:
get_infrastructure_stats_by_risk()

,risk_level,total_assets,avg_condition_score,total_maintenance_cost
0,critical,5,34.000000,3574319
1,high,183,52.715847,69603088
2,low,144,89.611111,66648744
3,medium,168,71.958333,69963923


In [81]:
get_infrastructure_stats_by_district()

,district,risk_level,total_assets,avg_condition_score,total_maintenance_cost
0,Business Bay,high,21,50.714286,5185214
1,Business Bay,low,12,92.333333,6663763
2,Business Bay,medium,14,72.642857,6482185
3,Deira,critical,2,40.000000,1564468
4,Deira,high,18,55.000000,12737126
5,Deira,low,15,92.466667,9903840
6,Deira,medium,22,71.818182,12639170
7,Downtown Dubai,high,14,54.428571,5586023
8,Downtown Dubai,low,20,87.450000,7910475
9,Downtown Dubai,medium,19,72.000000,5047267


In [84]:
!pip install 'strands-agents[ollama]' strands-agents-tools


In [108]:
from strands import Agent
from strands.models.ollama import OllamaModel

ollama_model = OllamaModel(
    host="http://localhost:11434",  # Ollama server address
    #model_id="MFDoom/deepseek-r1-tool-calling:1.5b",
    model_id="llama3.1:8b",
    think = False,
    reasoning = False
                                # Specil to use
)

/Users/mohamednoordeenalaudeen/Documents/Inceptez LangChain Classwork/Device Management Strands/venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3699: UserWarning: Invalid configuration parameters: ['reasoning', 'think'].
Valid parameters are: ['additional_args', 'keep_alive', 'max_tokens', 'model_id', 'options', 'stop_sequences', 'temperature', 'top_p'].

See https://github.com/strands-agents/sdk-python/issues/815
  exec(code_obj, self.user_global_ns, self.user_ns)


In [109]:
#Create an agent using the Ollama model
agent = Agent(model=ollama_model)

# Use the agent
agent("Tell me about Strands agents.") # Prints model output to stdout by default

Strands Agents are a fascinating topic!

Strands is a French company that specializes in artificial intelligence (AI) and personalization technologies, particularly in the field of digital identity and data management.

A Strands Agent is an AI-powered virtual assistant that uses machine learning algorithms to help users manage their financial lives. These agents can be integrated into various digital platforms, such as mobile apps, web applications, or even voice assistants like Alexa or Google Assistant.

Here's what you need to know about Strands Agents:

**Key Features:**

1. **Personalized advice**: Strands Agents use AI to analyze a user's financial data and provide personalized recommendations on budgeting, saving, and investing.
2. **Data aggregation**: These agents can collect and integrate financial data from various sources, such as bank accounts, credit cards, and investment portfolios.
3. **Goal-based planning**: Strands Agents help users set and achieve long-term financia

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Strands Agents are a fascinating topic!\n\nStrands is a French company that specializes in artificial intelligence (AI) and personalization technologies, particularly in the field of digital identity and data management.\n\nA Strands Agent is an AI-powered virtual assistant that uses machine learning algorithms to help users manage their financial lives. These agents can be integrated into various digital platforms, such as mobile apps, web applications, or even voice assistants like Alexa or Google Assistant.\n\nHere's what you need to know about Strands Agents:\n\n**Key Features:**\n\n1. **Personalized advice**: Strands Agents use AI to analyze a user's financial data and provide personalized recommendations on budgeting, saving, and investing.\n2. **Data aggregation**: These agents can collect and integrate financial data from various sources, such as bank accounts, credit cards, and investment p

In [110]:
agent = Agent(model=ollama_model)

# Use the agent
agent("my name is noordeen")

Hello Noordeen! It's nice to meet you. Where are you from? I'm here to help with any questions or topics you'd like to discuss. What brings you here today?

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Hello Noordeen! It's nice to meet you. Where are you from? I'm here to help with any questions or topics you'd like to discuss. What brings you here today?"}]}, metrics=EventLoopMetrics(cycle_count=1, tool_metrics={}, cycle_durations=[2.29762601852417], traces=[<strands.telemetry.metrics.Trace object at 0x12a61f930>], accumulated_usage={'inputTokens': 40, 'outputTokens': 16, 'totalTokens': 56}, accumulated_metrics={'latencyMs': 2281.954166}), state={})

In [111]:
agent("my name is noordeen")

I remember, your name is Noordeen!

Let's get started then! Would you like to:

1. Chat about a specific topic (e.g., hobbies, interests, news)?
2. Play a game or have some fun?
3. Ask for recommendations or advice on something?

Feel free to pick any of these options or let me know what's on your mind!

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "I remember, your name is Noordeen!\n\nLet's get started then! Would you like to:\n\n1. Chat about a specific topic (e.g., hobbies, interests, news)?\n2. Play a game or have some fun?\n3. Ask for recommendations or advice on something?\n\nFeel free to pick any of these options or let me know what's on your mind!"}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={}, cycle_durations=[2.29762601852417, 4.351131916046143], traces=[<strands.telemetry.metrics.Trace object at 0x12a61f930>, <strands.telemetry.metrics.Trace object at 0x12c3ba670>], accumulated_usage={'inputTokens': 117, 'outputTokens': 87, 'totalTokens': 204}, accumulated_metrics={'latencyMs': 6607.115331999999}), state={})

In [112]:
agent("what is my name?")

Your name is... Noordeen! (Or so you told me earlier!)

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'Your name is... Noordeen! (Or so you told me earlier!)'}]}, metrics=EventLoopMetrics(cycle_count=3, tool_metrics={}, cycle_durations=[2.29762601852417, 4.351131916046143, 1.184560775756836], traces=[<strands.telemetry.metrics.Trace object at 0x12a61f930>, <strands.telemetry.metrics.Trace object at 0x12c3ba670>, <strands.telemetry.metrics.Trace object at 0x12c130410>], accumulated_usage={'inputTokens': 134, 'outputTokens': 249, 'totalTokens': 383}, accumulated_metrics={'latencyMs': 7771.94954}), state={})